# 输出解析器

1. 普通解析器：`PydanticOutputParser`
2. 带有纠正的解析器 `OutputFixingParser` 
3. 带有纠正和补充内容功能的解析器 `RetryWithErrorOutputParser`

In [4]:
from langchain_community.chat_models import ChatTongyi

API_KEY = "sk-API_KEY"
model = ChatTongyi(model='qwen-turbo', top_p=0.8, temperature=0.1, api_key=API_KEY)

In [3]:
# ------Part 2
# 创建一个空的DataFrame用于存储结果
import pandas as pd
df = pd.DataFrame(columns=["flower_type", "price", "description", "reason"])

# 数据准备
flowers = ["玫瑰", "百合", "康乃馨"]
prices = ["50", "30", "20"]

# 定义我们想要接收的数据格式
from pydantic import BaseModel, Field
class FlowerDescription(BaseModel):
    flower_type: str = Field(description="鲜花的种类")
    price: int = Field(description="鲜花的价格")
    description: str = Field(description="鲜花的描述文案")
    reason: str = Field(description="为什么要这样写这个文案")


In [4]:

# ------Part 3
# 创建输出解析器
from langchain.output_parsers import PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=FlowerDescription)

# 获取输出格式指示
format_instructions = output_parser.get_format_instructions()
# 打印提示
print("输出格式：",format_instructions)


输出格式： The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"flower_type": {"description": "鲜花的种类", "title": "Flower Type", "type": "string"}, "price": {"description": "鲜花的价格", "title": "Price", "type": "integer"}, "description": {"description": "鲜花的描述文案", "title": "Description", "type": "string"}, "reason": {"description": "为什么要这样写这个文案", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}
```


In [5]:

# ------Part 4
# 创建提示模板
from langchain import PromptTemplate
prompt_template = """您是一位专业的鲜花店文案撰写员。
对于售价为 {price} 元的 {flower} ，您能提供一个吸引人的简短中文描述吗？
{format_instructions}"""

# 根据模板创建提示，同时在提示中加入输出解析器的说明
prompt = PromptTemplate.from_template(prompt_template, 
       partial_variables={"format_instructions": format_instructions}) 

# 打印提示
print("提示：", prompt)


提示： input_variables=['flower', 'price'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"flower_type": {"description": "鲜花的种类", "title": "Flower Type", "type": "string"}, "price": {"description": "鲜花的价格", "title": "Price", "type": "integer"}, "description": {"description": "鲜花的描述文案", "title": "Description", "type": "string"}, "reason": {"description": "为什么要这样写这个文案", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}\n```'} template='您是一位专业的鲜花店文案撰写员。\n对于售价为 {price} 元的 {flower}

In [8]:
# ------Part 5
for flower, price in zip(flowers, prices):
    # 根据提示准备模型的输入
    prompt_input = prompt.format(flower=flower, price=price)
    # 打印提示
    print("提示：", prompt_input)

    # 获取模型的输出
    output = model.invoke(prompt_input)

    # 解析模型的输出
    parsed_output = output_parser.parse(output.content)
    parsed_output_dict = parsed_output.dict()  # 将Pydantic格式转换为字典

    # 将解析后的输出添加到DataFrame中
    df.loc[len(df)] = parsed_output.dict()

# 打印字典
print("输出的数据：", df.to_dict(orient='records'))

提示： 您是一位专业的鲜花店文案撰写员。
对于售价为 50 元的 玫瑰 ，您能提供一个吸引人的简短中文描述吗？
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"flower_type": {"description": "鲜花的种类", "title": "Flower Type", "type": "string"}, "price": {"description": "鲜花的价格", "title": "Price", "type": "integer"}, "description": {"description": "鲜花的描述文案", "title": "Description", "type": "string"}, "reason": {"description": "为什么要这样写这个文案", "title": "Reason", "type": "string"}}, "required": ["flower_type", "price", "description", "reason"]}
```
提示： 您是一位专业的鲜花店文案撰写员。
对于售价为 30 元的 百合 ，您能提供一个吸引人的简短中文描述吗？
The output should be formatted as 

In [21]:
# 导入所需要的库和模块
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryWithErrorOutputParser
from pydantic import BaseModel, Field
from typing import List

# 使用Pydantic创建一个数据格式，表示花
class Flower(BaseModel):
    name: str = Field(description="name of a flower")
    colors: List[str] = Field(description="the colors of this flower")
# 定义一个用于获取某种花的颜色列表的查询
flower_query = "Generate the charaters for a random flower."

# 定义一个格式不正确的输出
misformatted = "{'name': '康乃馨', 'colors': ['粉红色','白色','红色','紫色','黄色']}"

# # 创建一个用于解析输出的Pydantic解析器，此处希望解析为Flower格式
parser = PydanticOutputParser(pydantic_object=Flower)
# # 使用Pydantic解析器解析不正确的输出
# parser.parse(misformatted)

# 使用OutputFixingParser创建一个新的解析器，该解析器能够纠正格式不正确的输出
new_parser = OutputFixingParser.from_llm(max_retries=2, parser=parser, llm=ChatTongyi(model='qwen-plus', top_p=0.8, temperature=0.1, api_key=API_KEY))
# 使用新的解析器解析不正确的输出
result = new_parser.parse(misformatted) # 错误被自动修正
print(result) # 打印解析后的输出结果


name='康乃馨' colors=['粉红色', '白色', '红色', '紫色', '黄色']


In [6]:
# 定义一个模板字符串，这个模板将用于生成提问
template = """Based on the user question, provide an Action and Action Input for what step should be taken.
{format_instructions}
Question: {query}
Response:"""

# 定义一个Pydantic数据格式，它描述了一个"行动"类及其属性
from pydantic import BaseModel, Field
class Action(BaseModel):
    action: str = Field(description="action to take")
    action_input: str = Field(description="input to the action")

# 使用Pydantic格式Action来初始化一个输出解析器
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser, RetryWithErrorOutputParser
parser = PydanticOutputParser(pydantic_object=Action)

# 定义一个提示模板，它将用于向模型提问
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_value = prompt.format_prompt(query="What are the colors of Orchid?")

# 定义一个错误格式的字符串
bad_response = '{"action": "search"}'
# parser.parse(bad_response) # 如果直接解析，它会引发一个错误

fix_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatTongyi(model='qwen-plus', top_p=0.8, temperature=0.1, api_key=API_KEY))
parse_result = fix_parser.parse(bad_response)
print('OutputFixingParser的parse结果:',parse_result)

retry_parser = RetryWithErrorOutputParser.from_llm(
    parser=parser, llm=ChatTongyi(model='qwen-plus', top_p=0.8, temperature=0.1, api_key=API_KEY)
)
parse_result = retry_parser.parse_with_prompt(bad_response, prompt_value)
print('RetryWithErrorOutputParser的parse结果:',parse_result)

OutputFixingParser的parse结果: action='search' action_input='query terms'
RetryWithErrorOutputParser的parse结果: action='search' action_input='colors of orchid'
